In [1]:
from google.colab import files

upload = files.upload()

Saving ai_news.pkl to ai_news.pkl
Saving amazon.pkl to amazon.pkl


In [3]:
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [18]:
with open('amazon.pkl','rb') as f:
  data = joblib.load(f)
locals().update(data)
print(tdm)
print(sentiment)

  (0, 942)	0.5663492253004965
  (0, 485)	0.5269561029030132
  (0, 884)	0.6336922128405843
  (1, 306)	0.3752776975307017
  (1, 146)	0.4710276120116065
  (1, 255)	0.47823718132718357
  (1, 912)	0.6392095406476104
  (2, 310)	0.4540368847647319
  (2, 355)	0.8909828883166824
  (3, 818)	0.4294194741822417
  (3, 155)	0.2986569556075369
  (3, 185)	0.3881684923718809
  (3, 368)	0.40528919670525015
  (3, 20)	0.4294194741822417
  (3, 407)	0.3548643091675774
  (3, 518)	0.3180236844892277
  (4, 310)	0.4666650388743726
  (4, 405)	0.8844341363224174
  (5, 485)	0.4240103899941458
  (5, 379)	0.478197306613838
  (5, 631)	0.4177530356423377
  (5, 205)	0.492450330030864
  (5, 922)	0.4177530356423377
  (6, 183)	0.719303976728933
  (6, 681)	0.6946954649786787
  :	:
  (993, 220)	1.0
  (994, 363)	1.0
  (995, 237)	0.27934807907261894
  (995, 225)	0.29240308948395255
  (995, 667)	0.34648694051732204
  (995, 266)	0.3960600022507811
  (995, 245)	0.3845968106507796
  (995, 747)	0.4536703935039088
  (995, 842)	0.45

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
#데이터 분할
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tdm, sentiment, test_size=0.2, random_state=1234)

In [0]:
#로지스틱 회귀 모형 만들기
from sklearn.linear_model import LogisticRegressionCV
model = LogisticRegressionCV()

In [8]:
#TDM을 활용한 지도 학습
model.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [9]:
model.score(x_train, y_train)

0.9625

In [10]:
model.score(x_test, y_test)

0.785

In [12]:
print(model.coef_.shape)
model.coef_[0,:10]


(1, 1000)


array([ 0.39804985,  0.40027349, -0.19645054, -0.34150143,  0.        ,
        0.19200845, -0.3389674 ,  0.23679498,  0.        ,  0.        ])

In [13]:
#감성사전 구축
#단어와 회귀계수 짝 만들기
import pandas as pd

sent_df = pd.DataFrame({'단어':vectorizer.get_feature_names(), '계수':model.coef_.flat})
sent_df.tail()

,단어,계수
995,wrongly,-0.251281
996,year,0.047634
997,years,1.442039
998,yell,-0.556269
999,yes,0.000000


In [19]:
#부정단어 사전
neg_df = sent_df[sent_df['계수']<0].sort_values(by='계수')
neg_df.head()

,단어,계수
492,poor,-2.563421
103,bad,-2.239719
803,terrible,-2.128419
988,worst,-1.892650
227,don,-1.884132


In [21]:
#긍정단어 사전
pos_df = sent_df[sent_df['계수']>0].sort_values(by='계수', ascending=False)
pos_df.head()

,단어,계수
310,great,5.266383
985,works,3.296928
114,best,3.265125
395,love,3.105271
255,excellent,3.062636


In [23]:
import numpy as np
#극성
sent_df['극성'] = np.sign(sent_df['계수'])
sent_df.set_index('단어',inplace=True)
sent_df.tail()

,계수,극성
단어,,
wrongly,-0.251281,-1.0
year,0.047634,1.0
years,1.442039,1.0
yell,-0.556269,-1.0
yes,0.000000,0.0


In [24]:
#계산 예제
sentence = 'poor screen but reasonable price'
words = sentence.split()
sent_df.reindex(words)

,계수,극성
단어,,
poor,-2.563421,-1.0
screen,0.175726,1.0
but,NaN,NaN
reasonable,0.297197,1.0
price,2.489607,1.0


In [25]:
sent = sent_df.reindex(words).dropna()
sent

sent['극성'].sum() / sent['극성'].abs().sum()

0.5